In [1]:
import pandas as pd
import csv
import obspy
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
csvfile = '/home/thompsong/src/Alexis_Montserrat_codes/MVOE_dailytraceid_wavfiles_df.csv'
df = pd.read_csv(csvfile)

In [2]:
print(df)

         Unnamed: 0  yyyymmdd         traceid  count
0                 0  19961001      .MBLY..PR2      0
1                 1  19961001  MV.MBGB.--.BHH      0
2                 2  19961001     .MBGE.J.SBE      0
3                 3  19961001      .MBGB..BHZ      0
4                 4  19961001  MV.MBRV.--.SHH      0
5                 5  19961001  MV.MBGB.--.BHZ      0
6                 6  19961001      .MBLY.2.PR      0
7                 7  19961001       .MBLG..AP      0
8                 8  19961001     .MBGB.J.SBN      0
9                 9  19961001      .MBLG.N.BH      0
10               10  19961001      .MBRY.Z.BH      0
11               11  19961001     .MBRY.J.S A      0
12               12  19961001  MV.MBLG.--.BHE      0
13               13  19961001      .MBGH.1.PR      0
14               14  19961001     .MBWH.J.A N      0
15               15  19961001      .MBLY.Z.SH      0
16               16  19961001  MV.MWHZ.--.SHZ      0
17               17  19961001  MV.MBRY.--.SHZ 

In [3]:
print(sorted(df.traceid.unique()))

['..J.SBN', '.DUMM.J.S E', '.DUMM.J.S N', '.DUMM.J.S Z', '.GUR3.I.SBE', '.GUR3.I.SBN', '.GUR3.I.SBZ', '.GUR4.I.SBE', '.GUR4.I.SBN', '.GUR4.I.SBZ', '.MBBE.I.SBE', '.MBBE.I.SBN', '.MBBE.I.SBZ', '.MBBE.J.SBE', '.MBBE.J.SBN', '.MBBE.J.SBZ', '.MBBY..BHE', '.MBBY..BHN', '.MBBY..BHZ', '.MBBY.E.BH', '.MBBY.J.SBE', '.MBBY.J.SBN', '.MBBY.J.SBZ', '.MBBY.N.BH', '.MBBY.Z.BH', '.MBBY.Z.SH', '.MBFL..SHZ', '.MBFL.Z.SH', '.MBFR..BHE', '.MBFR..BHN', '.MBFR..BHZ', '.MBFR.E.BH', '.MBFR.N.BH', '.MBFR.Z.BH', '.MBGA.J.SBE', '.MBGA.J.SBN', '.MBGA.J.SBZ', '.MBGB..BH', '.MBGB..BHE', '.MBGB..BHN', '.MBGB..BHZ', '.MBGB.E.BH', '.MBGB.J.SBE', '.MBGB.J.SBN', '.MBGB.J.SBZ', '.MBGB.N.BH', '.MBGB.Z.BH', '.MBGE.J.SBE', '.MBGE.J.SBN', '.MBGE.J.SBZ', '.MBGH..BH', '.MBGH..BHE', '.MBGH..BHN', '.MBGH..BHZ', '.MBGH.1.PR', '.MBGH.2.PR', '.MBGH.E.BH', '.MBGH.J.SBE', '.MBGH.J.SBN', '.MBGH.J.SBZ', '.MBGH.N.BH', '.MBGH.Z.BH', '.MBHA..BHE', '.MBHA..BHN', '.MBHA..BHZ', '.MBHA.E.BH', '.MBHA.N.BH', '.MBHA.Z.BH', '.MBLG..AP', '.MBLG..A

In [ ]:
def translate_id2site(id):
    thissite = id[4:6]
    if id[3:7]=='LONG':
        thissite = 'LG'
    if id[3:5]=='MW':
        thissite = 'WH'
    if id[3:6]=='MVO':
        thissite = 'MVO'
    if id[4:7]=='NEV':
        thissite = 'NEV'
    if id[3:5]=='MP':
        thissite = 'PL'
    if id[3:6]=='MCH':
        thissite = 'CP'
    if id[3:7]=='IRIG' or id[3:7]=='TST1' or id[3:6]=='MRH':
        thissite = ''
    return thissite    

In [ ]:
uniqids = df.traceid.unique()
sites = set()
for id in uniqids:
    thissite = translate_id2site(id)
    if not thissite:
        continue
    sites.add(thissite)
sites = sorted(sites)
print(sites)

In [ ]:
# loop over rows in df, and create a new dataframe for sites, with a 1 to indicate if on for each day
sites_df = pd.DataFrame(columns = ['date'] + sorted(sites))
for index, row in df.iterrows():
    thisdate = row['yyyymmdd']
    thissite = translate_id2site(row['traceid'])
    if not thissite:
        continue
    thiscount = row['count']
    if np.isnan(thiscount):
        thiscount = 0
    currentcount = 0
    try:
        currentcount = sites_df.at[thisdate,thissite]
    except:
        pass
    if np.isnan(currentcount):
        currentcount = 0
    if thiscount > currentcount:
        sites_df.at[thisdate,thissite] = thiscount
print(sites_df)

In [ ]:
sites_bool_df = (sites_df > 0) * 1

In [ ]:
data = np.array(sites_bool_df, dtype=float) # convert dataframe to numpy array
data = 1-data[:,1:] # remove 'date' column which for some reason is empty
print(data.shape)

In [ ]:
# Set labels for ontime plot
xticklabels = sites
yticklabels = []
yticks=[]
i = 0
for index, row in sites_bool_df.iterrows():
    date = str(index)
    if date[-2:]=='01':
        yticklabels.append(date)
        yticks.append(i)
    i+=1
ystep = 1
if len(yticks)>15:
    ystep=2
if len(yticks)>25:
    ystep=3
if len(yticks)>40:
    ystep=4
if len(yticks)>60:
    ystep=6
if len(yticks)>120:
    ystep=12    
yticks = yticks[0::ystep]
yticklabels = yticklabels[0::ystep]
print(yticklabels)

In [ ]:
# make ontime plot
fig = plt.figure()
ax = fig.add_subplot(111)
ax.imshow(data, aspect='auto', cmap=plt.cm.gray, interpolation='nearest')
plt.xticks(np.arange(len(xticklabels)), xticklabels)
ax.set_xticklabels(xticklabels, rotation = 90)
plt.yticks(yticks, yticklabels)
plt.tight_layout()
plt.savefig('ASNE_siteontime.png',dpi=200)
plt.ylabel('Date')
plt.xlabel('Site')

In [ ]:
# make number of stations operational per day plot
fig = plt.figure()
ax = fig.add_subplot(111)
s = np.sum(1-data,axis=1)
plt.plot(s,'.')
plt.xticks(yticks, yticklabels)
ax.set_xticklabels(yticklabels, rotation = 90)
plt.ylabel('Number of sites operational')
plt.savefig('ASNE_numsitesperday.png',dpi=200)

In [ ]:
# make number of WAVfiles per day plot
fig = plt.figure()
ax = fig.add_subplot(111)
data2 = np.array(sites_df) # convert dataframe to numpy array
data2 = data2[:,1:] # remove 'date' column which for some reason is empty
print(data2.shape)
y = np.nanmax(data2,axis=1)
print(y.shape)
plt.plot(y)
plt.xticks(yticks, yticklabels)
ax.set_xticklabels(yticklabels, rotation = 90)
plt.ylabel('Number of WAV files per day')
#plt.ylim(-5,10)
plt.savefig('ASNE_numwavfilesperday.png',dpi=200)

In [ ]:
print(np.sum(y))